# INF264 - Project 1: Implementing Decision Trees
=========================================================================

A machine learning project for implementing decision trees as part of the INF264 course.

## Table of Contents

- [0.1 Imports and Data Loading](#0.1-Imports-and-Data-Loading)
- [0.2 Data Analysis and Preprocessing](#0.2-Data-Analysis-and-Preprocessing)
- [1.1 Implement a Decision Tree Learning Algorithm, From Scratch](#1.1-Implement-a-Decision-Tree-Learning-Algorithm,-From-Scratch)
- [1.2 Add Gini Index](#1.2-Add-Gini-Index)
- [1.3 Add Reduced-Error Pruning](#1.3-Add-Reduced-Error-Pruning)
- [1.4 Evaluate Your Algorithm](#1.4-Evaluate-Your-Algorithm)
- [1.5 Compare to an Existing Implementation](#1.5-Compare-to-an-Existing-Implementation)

## 0.1 Imports and Data Loading 

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from time import time

# Load the dataset from a CSV file into a DataFrame.
data = pd.read_csv("wine_dataset.csv")

## 0.2 Data Analysis and Preprocessing

In [2]:
# Split the dataset into features and labels.
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [3]:
data.head()

,citric acid,residual sugar,pH,sulphates,alcohol,type
0,0.13,1.60,3.34,0.59,9.2,1
1,0.10,2.80,3.60,0.66,10.2,1
2,0.32,1.90,3.20,0.55,9.5,1
3,0.29,13.65,3.00,0.60,9.5,0
4,0.26,2.00,3.41,0.74,9.2,1


The dataset consists of the features: 'citric acid', 'residual sugar', 'pH', 'sulphates', and 'alcohol', and the binary target 'type'.

In [4]:
# Check the types of each feature/label.
data.dtypes

citric acid       float64
residual sugar    float64
pH                float64
sulphates         float64
alcohol           float64
type                int64
dtype: object

The features are encoded as floats and the target is encoded as an integer.

In [5]:
# Check for missing values.
data.isnull().sum()

citric acid       0
residual sugar    0
pH                0
sulphates         0
alcohol           0
type              0
dtype: int64

The dataset does not contain any missing values.

In [6]:
data.describe()

,citric acid,residual sugar,pH,sulphates,alcohol,type
count,3198.000000,3198.000000,3198.000000,3198.000000,3198.000000,3198.000000
mean,0.301776,4.449781,3.249678,0.574431,10.459725,0.500000
std,0.165284,4.214445,0.163439,0.165587,1.143231,0.500078
min,0.000000,0.600000,2.740000,0.220000,8.000000,0.000000
25%,0.210000,1.900000,3.140000,0.470000,9.500000,0.000000
50%,0.300000,2.400000,3.240000,0.550000,10.200000,0.500000
75%,0.400000,5.937500,3.360000,0.650000,11.200000,1.000000
max,1.660000,65.800000,4.010000,2.000000,14.900000,1.000000


There might be some outliers. For example in 'residual sugar' there are a big difference between the max and 75%, which may indicate outliers. This does not matter since decision trees are not sensitive to outliers. 

The dataset is not normalized because the features are not between 0 and 1. Normalizations is also not needed for decision trees because they make their decisions (splits) on one feature at a time. Meaning they are not sensitive to variance in the dataset. 

In [7]:
# Check the proportions between each label.
y.value_counts(normalize=True)

type
1    0.5
0    0.5
Name: proportion, dtype: float64

The dataset is perfectly balanced as all things should be.

The dataset has been split into features (X) and target/labels (y). And will later be split into training, validation, testing and pruning sets.

# 1.1 Implement a Decision Tree Learning Algorithm, From Scratch

In [8]:
class DecisionNode:
    """Represents a node in a decision tree.
    
    The node stores the criteria (feature and threshold) for splitting the data into two children.
    It recursively directs the data point left or right to predict.
    
    Attributes:
        feature_index (int): Index of the feature used for splitting.
        threshold (float): The threshold value used for splitting.
        left (DecisionNode or Leaf): Left child (subtree) for values <= threshold.
        right (DecisionNode or Leaf): Right child (subtree) for values > threshold.
    
    """
    
    def __init__(self, feature_index, threshold, left, right):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
    
    def predict(self, x):
        """Predict label for the given data point.
        
        Recursively goes though the decision tree, based on the features value and the corresponding threshold.
        
        Args:
            x (Series): A single data point.
            
        Returns:
            int: Predicted class label.
        
        """
        
        # If the value of the feature at index 'feature_index' for the datapoint x 
        # is less than or equal to the threshold value, go left.
        if x[self.feature_index] <= self.threshold:
            return self.left.predict(x)
        # Else, go right. 
        else:
            return self.right.predict(x)

class Leaf:
    """Represents a leaf node in a decision tree.
    
    The leaf contains the final predicted label, and has no children.
    
    Attributes:
        label (int or str): Predicted label.
    
    """
    
    def __init__(self, label):
        self.label = label
    
    def predict(self, x):
        """Return the predicted label stored in this leaf.
        
        Args:
            x (Series): A single data point (not used in the method, but included for consistency).
        
        Returns:
            int or str: Predicted label.
        
        """
        return self.label

In [9]:
def identical_labels(y):
    """Check if all labels are identical.
    
    Args:
        y (Series): Series containing labels.
        
    Returns:
        bool: True if all labels are identical, False if not.
    
    """
    
    return y.nunique() == 1

In [10]:
def identical_features(X):
    """Check if all values within each feature column are identical.
    
    Args:
        X (DataFrame): DataFrame containing features.
        
    Returns:
        bool: True if all values within each feature column are identical, False if not.
    
    """
    
    return (X.nunique() == 1).all()

In [11]:
def entropy(y):
    """Calculate the entropy of given labels.
    
    Args:
        y (Series): Series of labels.
    
    Returns:
        float: The entropy value for the provided labels.
    
    """
    
    # Get the counts of all the labels
    labels, counts = np.unique(y, return_counts=True)
    
    # Calculate the total number of labels
    total_labels = len(y)
    
    # Calculate entropy
    probabilities = counts / total_labels
    return -np.sum(probabilities * np.log2(probabilities))

In [12]:
def information_gain(y, y_left, y_right, impurity_measure='entropy'):
    """Calculate the information gain.
    
    The information gain is the reduction in uncertainty (uncertainty before split - uncertainty after split).
    
    Args:
        y (Series): Series containing labels.
        y_left (Series): Series containing labels for the left split.
        y_right (Series): Series containing labels for the right split.
        impurity_measure (str, optional): The measure used to determine which impurity measure to use.
            Default is 'entropy'.
    
    Returns:
        float: The information gain value for the provided split. 
        
    Raises:
        ValueError: If the provided impurity_measure is not in the impurity_measures dictionary.
    
    """
    
    # Dictionary holding the different impurity measures.
    impurity_measures = {
        'entropy': entropy,
        'gini': gini
    }
    
    # Raise an error if the given impurity_measure is not in the impurity_measures dictionary.
    if impurity_measure not in impurity_measures:
        raise ValueError(f'Invalid impurity_measure: \'{impurity_measure}\'. Available impurity measures: {list(impurity_measures.keys())}')
    
    # Calculate the original impurity.
    original_impurity = impurity_measures[impurity_measure](y)
    
    # Calculate the impurity for the "left" and "right" split.
    left_measure = impurity_measures[impurity_measure](y_left)
    right_measure = impurity_measures[impurity_measure](y_right)
    
    # Calculate the proportions of data points in the left and right splits    
    prop_left = len(y_left) / len(y)
    prop_right = len(y_right) / len(y)
    
    # Calculate the new (average weighted) impurity measure
    new_impurity = prop_left * left_measure + prop_right * right_measure
    
    # Return the information gain
    return original_impurity - new_impurity

In [13]:
def build_tree(X_train, y_train, impurity_measure):
    """Recursively build a decision tree based on the provided data and impurity measure.
    
    Args:
        X_train (DataFrame): DataFrame containing training features.
        y_train (Series): Series containing training labels.
        impurity_measure (str, optional): The measure used to determine which impurity measure to use.
            Default is 'entropy'.
            
    Returns:
        Node: A DecisionNode (if there are a valid split), or a Leaf.
        
    Raises:
        ValueError: If X_train and y_train are empty or different sizes.
    
    """
        
    # Raise an error if the given X and y is empty or different sizes. 
    if X_train.shape[0] == 0 or y_train.shape[0] == 0:
        raise ValueError('X or y is empty.')
    if X_train.shape[0] != y_train.shape[0]:
        raise ValueError('X and y is not the same size.')
    
    # Check if all the labels are the same. If true, return a Leaf node with that label.
    if identical_labels(y_train):
        return Leaf(y_train.iloc[0])

    # Check if all features values across all instances are the same.
    # If True, get the most common value (if multiple labels are equally common, the first one is chosen).
    # Return a Leaf node with the most common label. 
    elif identical_features(X_train):
        most_common_label = y_train.mode().iloc[0]
        return Leaf(most_common_label)
    
    # Set initial variables to 0 and None. 
    best_gain = 0
    best_split = None
    best_left_indices = None
    best_right_indices = None

    # For each feature in the X_train:
    for feature in range(X_train.shape[1]):
        # Get the feature values from the current feature in X_train.
        values = X_train.iloc[:, feature]
        
        # Calculate the median of the feature values. 
        median_value = values.median()
        
        # Split the feature indicies based on the median value. 
        left_indices = np.where(values < median_value)[0]
        right_indices = np.where(values >= median_value)[0]
        
        # Get the labels from the splits using the indices.
        y_train_left = y_train.iloc[left_indices]
        y_train_right = y_train.iloc[right_indices]
        
        # Get the information gain for the split.
        gain = information_gain(y_train, y_train_left, y_train_right, impurity_measure)

        # If the current split is better than the previous best split, update the variables.
        if gain > best_gain:
            best_gain = gain
            best_split = feature, median_value
            best_left_indices = left_indices
            best_right_indices = right_indices

    # If the best gain is 0, there is no benefit to split. Return a leaf with the most common value.
    if best_gain == 0:
        return Leaf(y_train.value_counts().idxmax())

    # Recursively create the left and right children (subtrees) using the best split.
    left_tree = build_tree(X_train.iloc[best_left_indices, :], y_train.iloc[best_left_indices], impurity_measure)
    right_tree = build_tree(X_train.iloc[best_right_indices, :], y_train.iloc[best_right_indices], impurity_measure)
    
    # Return a new decision node with the best split feature, threshold (median) and the left and right subtree.
    return DecisionNode(best_split[0], best_split[1], left_tree, right_tree)

In [14]:
def learn(X, y, impurity_measure='entropy', prune=False, pruning_set_size=0.1, pruning_threshold=0):
    """Trains a decision tree using the provided X and y. Optionally prunes the tree.

    Args:
        X (DataFrame): The dataframe containing the features.
        y (Series): The labels corresponding to each sample in X.
        impurity_measure (str, optional): The measure used to determine which impurity measure to use.
            Default is 'entropy'.
        prune (bool, optional): Whether to prune the tree or not. 
            Default is False.
        pruning_set_size (float, optional): If pruning is enabled, this is the ratio of the dataset 
            to use for the pruning set. Default is 0.1 (10%).
        pruning_threshold (float): The amount that the accuracy need to improve before we prune.
            Default is 0.

    Returns:
        DecisionNode: The root node of the decision tree.

    Raises:
        ValueError: If `X` or `y` is empty, or if they are not the same size.
        
    """
    
    # Raise an error if the given X and y is empty or different sizes. 
    if X.shape[0] == 0 or y.shape[0] == 0:
        raise ValueError('X or y is empty.')
    if X.shape[0] != y.shape[0]:
        raise ValueError('X and y is not the same size.')
    
    # If prune is true:
    if prune:
        # Split the data into training and pruning sets.
        X_train, X_prune, y_train, y_prune = train_test_split(X, y, test_size=pruning_set_size, random_state=42)
        
        # Build the decision tree.
        tree = build_tree(X_train, y_train, impurity_measure)
        
        # Prune the tree and return it.
        pruned_tree = prune_tree(tree, X_train, y_train, X_prune, y_prune, pruning_threshold)
        return pruned_tree
    
    # If prune is false, build and return the decision tree.
    else:
        tree = build_tree(X, y, impurity_measure)
        return tree

In [15]:
def tree_stats(tree):
    """Compute total number of nodes, leaves and max depth of a given tree.
    
    Args:
        tree (DecisionNode or Leaf): Root node of a tree.
    
    Returns:
        tuple:
            - maximum depth
            - total number of leaves
            - total number of nodes
            
    """
    
    if isinstance(tree, Leaf):
        return 0, 1, 1 # depth, leaf, node
    
    # Recursively compute the depth, number of leaves and number of nodes for left and right subtree.
    left_depth, left_leaves, left_nodes = tree_stats(tree.left)
    right_depth, right_leaves, right_nodes = tree_stats(tree.right)
    
    # Get the max depth of left and right subtree.
    max_depth = 1 + max(left_depth, right_depth)
    
    # Sum total number of leaves in the left and right subtree to get the total number of leaves. 
    total_leaves = left_leaves + right_leaves 
    
    # Sum total number of nodes in the left and right subtree and add 1 (the current node) to get total number of nodes.
    total_nodes = 1 + left_nodes + right_nodes
    
    return max_depth, total_leaves, total_nodes

In [16]:
def print_tree_stats(tree):
    """ Compute and print the statistics of a given decision tree.
    
    Args:
        node (DecisionNode or Leaf): The root node of a decision tree.
    
    """
    
    max_depth, total_leaves, total_nodes = tree_stats(tree)
    print(f'Number of nodes: {total_nodes}')
    print(f'Number of leaf nodes: {total_leaves}')
    print(f'Depth of the tree: {max_depth}')

# 1.2 Add Gini Index

In [17]:
def gini(y):
    """Calculate the Gini of given labels.
    
    Args:
        y (Series): Series of labels
        
    Returns:
        float: The Gini value for the provided labels.
    
    """
    
    # Get the counts of all the labels
    labels, counts = np.unique(y, return_counts=True)
    
    # Calculate the total number of labels
    total_labels = len(y)
    
    # Calculate Gini
    probabilities = counts / total_labels
    return 1-np.sum(probabilities ** 2)

# 1.3 Add Reduced-Error Pruning

In [18]:
def get_predictions(node, X):
    """Get predictions for a dataset using given node.
    
    Args:
        node (DecisionNode or Leaf): The node used for prediction.
        X (DataFrame): The dataset.
        
    Returns:
        list: A list of predictions for each row in the DataFrame.
    
    """
    
    predictions = []
    
    # Iterates through each row in the DataFrame X.
    for _, x in X.iterrows():
        # Predict the label for the current row using the provided node.
        prediction = node.predict(x)
        predictions.append(prediction)
    return predictions

In [19]:
def prune_tree(node, X_train, y_train, X_prune, y_prune, pruning_threshold=0):
    """Recursively prune the decision tree.
    
    Args:
        node (DecisionNode or Leaf): The original (pre-pruning) node or child.
        X_prune (DataFrame): The features of the pruning dataset.
        y_prune (Series): The labels of the pruning dataset.
        pruning_threshold (float): The amount that the accuracy need to improve before we prune.
            Default is 0.
    
    Returns:
        DecisionNode or Leaf: The pruned node or child.
        
    """
    
    # Check if it is a leaf node. If it is, no pruning is needed (there is nothing to prune). 
    if isinstance(node, Leaf):
        return node
    
    # If it's a DecisionNode, prune its children. This makes it recursive, and makes it a buttom-up approach.
    node.left = prune_tree(node.left, X_train, y_train, X_prune, y_prune, pruning_threshold=pruning_threshold)
    node.right = prune_tree(node.right, X_train, y_train, X_prune, y_prune, pruning_threshold=pruning_threshold)

    # Get the column name from the feature index
    feature_name = X_prune.columns[node.feature_index]
    
    # Filter the pruning dataset based on the decision at the current node
    left_filter = X_prune[feature_name] <= node.threshold
    right_filter = X_prune[feature_name] > node.threshold

    left_data = y_prune[left_filter]
    right_data = y_prune[right_filter]

    # Make a prediction with the original tree structure.
    original_predictions = get_predictions(node, X_prune)
        
    # Calculate the accuracy before pruning.    
    original_accuracy = sum(original_predictions == y_prune) / len(y_prune)
    
    # Find the majority class of the labels in the pruning set. Create a new leaf node with this class.
    majority_class = pd.concat([y_prune[left_filter], y_prune[right_filter]]).mode().iloc[0]
    leaf_node = Leaf(majority_class)
    
    # Make a prediction using the pruned child.
    pruned_predictions = get_predictions(leaf_node, X_prune)
    
    # Calculate the accuracy after pruning (the accuracy if the original node is replaced by the new leaf node).
    pruned_accuracy = sum(pruned_predictions == y_prune) / len(y_prune)
    
    # If replacing the child with the new leaf node does not decrease accuracy, replace the child with the leaf node.
    if pruned_accuracy >= original_accuracy + pruning_threshold:
        return leaf_node
    else:
        return node

# 1.4 Evaluate Your Algorithm

In [20]:
# Split the data into X (features) and y (labels).
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Split the data into a temporary and a training test set.
# Then split the temporary set into validation and testing set.
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [21]:
X_train

,citric acid,residual sugar,pH,sulphates,alcohol
3034,0.38,8.1,3.30,0.54,9.8
2576,0.23,6.2,3.34,0.43,9.6
533,0.04,2.5,3.53,0.55,9.5
1061,0.20,3.0,3.23,0.59,9.5
2626,0.32,16.2,3.17,0.37,11.2
...,...,...,...,...,...
1095,0.59,11.8,3.17,0.46,8.9
1130,0.30,1.2,2.96,0.36,12.5
1294,0.00,2.2,3.40,0.58,10.9
860,0.14,2.4,3.66,0.65,9.8


### 1.4.1 Model Training and Selection


In [22]:
# List of impurity measures and pruning settings (with or without) to test.
settings_measures = ['entropy', 'gini']
settings_pruning = [True, False]

# Variables to keep track of the best performing tree, impurity measure, prune setting and accuracy.
best_tree = None
best_measure = 0
best_prune_setting = False
best_accuracy = 0

print('-' * 35)

# Loop through each impurity measure with each prune setting. 
for measure in settings_measures:    
    for prune in settings_pruning:
        # Train the decision tree using the current settings.
        tree = learn(X_train, y_train, impurity_measure=measure, prune=prune)
        
        # Make a prediction using validation data.
        predictions = get_predictions(tree, X_val)
        
        # Calculate the accuracy of the predictions. 
        accuracy = sum(predictions == y_val) / len(y_val)
        
        # Calculate the F1 score of the predictions.
        f1 = f1_score(y_val, predictions)
        
        # If accuracy improves, update best_measure, best_accuracy, best_prune_setting and best_tree
        # to the current values.
        if accuracy > best_accuracy:
            best_measure = measure
            best_accuracy = accuracy
            best_prune_setting = prune
            best_tree = tree
        
        # Print the results with the current settings.
        pruning_status = 'with pruning' if prune else 'without pruning'
        print(f'\033[1m{measure.capitalize()} {pruning_status}\033[0m ')
        print(f'Accuracy: {accuracy:.4f}')
        print(f'F1 score: {f1:.4f}')
        print_tree_stats(tree)
        print('-' * 35)

# Print the best model and the result.
pruning_status = 'with pruning' if best_prune_setting else 'without pruning'        
print(f'\033[1mBest model: {best_measure.capitalize()} {pruning_status}\033[0m')
print('-' * 35)

-----------------------------------
Entropy with pruning 
Accuracy: 0.7906
F1 score: 0.8154
Number of nodes: 375
Number of leaf nodes: 188
Depth of the tree: 12
-----------------------------------
Entropy without pruning 
Accuracy: 0.8562
F1 score: 0.8477
Number of nodes: 989
Number of leaf nodes: 495
Depth of the tree: 13
-----------------------------------
Gini with pruning 
Accuracy: 0.7688
F1 score: 0.8000
Number of nodes: 361
Number of leaf nodes: 181
Depth of the tree: 11
-----------------------------------
Gini without pruning 
Accuracy: 0.8781
F1 score: 0.8713
Number of nodes: 977
Number of leaf nodes: 489
Depth of the tree: 13
-----------------------------------
Best model: Gini without pruning
-----------------------------------


* Entropy with pruning vs without pruning:
    - Accuracy with pruning ≈ 0.79
    - Accuracy without pruning ≈ 0.86
    
The tree using entropy performs better without pruning.

* Gini with pruning vs without pruning:
    - Accuracy with pruning ≈ 0.77
    - Accuracy without pruning ≈ 0.88
    
The tree using gini also performs better without pruning.

* Entropy without pruning vs Gini without pruning:
    - Accuracy entropy: 0.86
    - Accuracy Gini: 0.88
    
Both trees perform similarly, with Gini without pruning performing the best at 88% accuracy.

### 1.4.2 Model Evaluation

In [23]:
# Make predictions on the test set using the best model found in 1.4.1.
test_predictions = get_predictions(best_tree, X_test)

# Calculate the accuracy of the predictions.
test_accuracy = sum(test_predictions == y_test) / len(y_test)

# Calculate the F1 score of the predictions.
test_f1 = f1_score(y_val, predictions)

# Print the test set accuracy using the best model.
print('-' * 40)
pruning_status = 'with pruning' if best_prune_setting else 'without pruning'
print(f'\033[1m{best_measure.capitalize()} {pruning_status} on test dataset\033[0m ')
print(f'Accuracy: {test_accuracy:.4f}')
print(f'F1 score: {test_f1:.4f}')
# Print number of nodes, leaf nodes, and depth.
print_tree_stats(tree)
print('-' * 40)

----------------------------------------
Gini without pruning on test dataset 
Accuracy: 0.8375
F1 score: 0.8713
Number of nodes: 977
Number of leaf nodes: 489
Depth of the tree: 13
----------------------------------------


The decision tree with Gini and without pruning had an accuracy of 0.84 on the test set. This is a more accurate estimate of real-world performance because the test set is previously unseen data.

### 1.4.3 Sanity Check

In [24]:
# Check if the outputs are constant (if all predictions are the same).
if len(set(test_predictions)) == 1:
    print('The outputs are constant.')
else:
    print('The outputs are not constant.')

The outputs are not constant.


# 1.5 Compare to an Existing Implementation
### 1.5.1 Training a Sklearn Decision Tree Classifier

In [25]:
# Create a instance of the decision tree classifier.
clf = DecisionTreeClassifier(random_state=42)

# Train the decision tree classifier, save the start time and end time, and calculate training time.
start_time_sklearn = time()
clf.fit(X_train, y_train)
end_time_sklearn = time()
train_time_sklearn = end_time_sklearn - start_time_sklearn

### 1.5.2 Training the Custom Decision Tree

In [26]:
# Train the custom decision tree, save the start time and end time, and calculate train time. 
# Using the best parameters found in 1.4.1.
start_time_custom = time()
custom_tree = learn(X_train, y_train, impurity_measure=best_measure, prune=best_prune_setting)
end_time_custom = time()
train_time_custom = end_time_custom - start_time_custom

### 1.5.3 Make Predictions and Compare

In [27]:
# Predict on test set for both models.
predictions_sklearn = clf.predict(X_test)
predictions_custom = get_predictions(custom_tree, X_test)

# Calculating accuracy for both models.
accuracy_sklearn = accuracy_score(y_test, predictions_sklearn)
accuracy_custom = accuracy_score(y_test, predictions_custom)

# Calculate the F1 score for both models.
f1_sklearn = f1_score(y_test, predictions_sklearn)
f1_custom = f1_score(y_test, predictions_custom)

print('-' * 45)
print(f'\033[1mSklearn\'s DecisionTreeClassifier\033[0m ')
print(f"Accuracy: {accuracy_sklearn:.4f}")
print(f"F1-score: {f1_sklearn:.4f}")
print(f"Training time: {train_time_sklearn:.4f} seconds")
# Print number of nodes, leaf nodes, and depth.
print(f"Number of nodes: {clf.tree_.node_count}")
print(f"Number of leaf nodes: {clf.tree_.n_leaves}")
print(f"Depth of the tree: {clf.tree_.max_depth}")

print('-' * 45)
print(f'\033[1mCustom Decision Tree\033[0m ')
print(f"Accuracy: {accuracy_custom:.4f}")
print(f"F1-score: {f1_custom:.4f}")
print(f"Training time: {train_time_custom:.4f} seconds")
# Print number of nodes, leaf nodes, and depth.
print_tree_stats(custom_tree)

print('-' * 45)
print(f'\033[1mComparison (Sklearn vs Custom)\033[0m ')
print(f"Accuracy difference: {(accuracy_custom - accuracy_sklearn):.4f}")
print(f"F1-score difference: {(f1_custom - f1_sklearn):.4f}")
print(f"Training time difference: {(train_time_custom - train_time_sklearn):.4f} seconds")
print('-' * 45)

---------------------------------------------
Sklearn's DecisionTreeClassifier 
Accuracy: 0.9000
F1-score: 0.9006
Training time: 0.0053 seconds
Number of nodes: 501
Number of leaf nodes: 251
Depth of the tree: 18
---------------------------------------------
Custom Decision Tree 
Accuracy: 0.8375
F1-score: 0.8344
Training time: 0.9201 seconds
Number of nodes: 977
Number of leaf nodes: 489
Depth of the tree: 13
---------------------------------------------
Comparison (Sklearn vs Custom) 
Accuracy difference: -0.0625
F1-score difference: -0.0662
Training time difference: 0.9148 seconds
---------------------------------------------


The custom decision tree has a 6% lower accuracy than Sklearn's DesicionTreeClassifier. The custom decision tree also takes longer time to train, and has almost the double amount of nodes and leaves. Sklear's DecisionTreeClassifier is deeper. This might suggest that the custom decision tree is either overfitting or using an unoptimized approach to split the data. 

### 1.5.4 Observations

* Accuracy
    - The custom decision tree has an accuracy of 84%. This is lower than sklearn's accuracy of 90%.
    - It might be small differences because of randomness, the rest might be because of a more optimized method to split features. 
* Speed
    - Sklearn is around 150 times faster, and that is with the custom tree not using pruning (which would be even slower). This suggests that sklearn's implementation is a lot more optimized than the custom tree.
* Overall
    - Both models are performing decent, but Sklearn's model is a bit better at both accuracy and speed. While the speed does not matter on datasets of this size, it might matter on larger datasets. 